In [ ]:
#  Libraries
import tqdm
import numpy as np 
import pandas as pd
from time import time
import datetime as dt
import pickle
import gc
# Data processing, metrics and modeling
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from bayes_opt import BayesianOptimization
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc,precision_recall_curve
from sklearn import metrics
from sklearn import preprocessing
# Lgbm
import lightgbm as lgb
# Suppr warning
import warnings
warnings.filterwarnings("ignore")
import json

import itertools
from scipy import interp

# Plots
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams

In [ ]:
#import in dataset
f = open("features_optimized.txt","r")
features_list = []
features_list = f.read().splitlines()
f.close()
print (features_list)

train = pd.read_csv("train_prepared.csv")
test = pd.read_csv("test_prepared.csv")

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
#encode the dataframe
for col in tqdm.tqdm(train.columns): 
    if train[col].dtype == 'object':
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))  

In [ ]:
#reduce mem usage
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
#isolate test and traindata
test_data = test[features_list]
train_data = train[features_list]

In [ ]:
result_list = test["TransactionID"].to_frame(name="TransactionID")
mydf = train['isFraud']
target = mydf

In [ ]:
del(test)
del(train)
gc.collect()

In [ ]:
#create lgb dataset from train set
# d_train = lgb.Dataset(train_data, label=target)

In [ ]:
# #Declare params here
#highest score
params= {'num_leaves': 250,
          'min_child_weight': 0.1,
          'feature_fraction': 0.5,
          'bagging_fraction': 0.5,
          'min_data_in_leaf': 256,
          'bagging_freq': 5,
          'objective': 'binary',
          'max_depth': -1, 
          'learning_rate': 0.001,
          "boosting_type": "gbdt", 
          "bagging_seed": 4041,   
          "feature_fraction_seed":4041, 
          "metric": 'auc',       
          'reg_alpha': 0.,        
          'reg_lambda': 2.,      
         } 
#change to num leaves
params3= {'num_leaves': 250,
          'min_child_weight': 0.1,
          'feature_fraction': 0.5,
          'bagging_fraction': 0.5,
          'min_data_in_leaf': 256,
          'bagging_freq': 5,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.1,
          "boosting_type": "gbdt",
          "bagging_seed": 4041,
          "feature_fraction_seed":4041,
          "metric": 'auc',
          'reg_alpha': 1.,
          'reg_lambda': 2.,
         }


paramList = [params]

In [ ]:
#catboost params here
#paramList = [params,params1,params2,params3,params4]

In [ ]:
#prepare folds for validation
folds = StratifiedKFold(n_splits=5,shuffle=False)

In [ ]:
def callTraining(index,train_data,target,folds,params):
    aucs_list = list()
    feature_ranking = pd.DataFrame()
    feature_ranking['feature'] = train_data.columns
    max_rds = []
    training_start_time = time()
    for fold,(trn_idx,test_idx) in enumerate(folds.split(train_data,target)):
        start_time = time()
        print('Training on fold {}'.format(fold+1))
        trn_data = lgb.Dataset(train_data.iloc[trn_idx], label=target.iloc[trn_idx])
        val_data = lgb.Dataset(train_data.iloc[test_idx], label=target.iloc[test_idx])
        lgb_clf = lgb.train(params, trn_data, 25000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=100)
        feature_ranking['fold_{}'.format(fold + 1)] = lgb_clf.feature_importance()
        aucs_list.append(lgb_clf.best_score['valid_1']['auc'])
        max_rds.append(lgb_clf.best_iteration)
    print('Fold {} finished in {}'.format(fold + 1, str(dt.timedelta(seconds=time() - start_time))))
    print('-' * 30)
    print('Training has finished.')
    print('Total training time is {}'.format(str(dt.timedelta(seconds=time() - training_start_time))))
    print('Mean AUC:', np.mean(aucs_list))
    print('-' * 30)
    
    feature_ranking['average'] = feature_ranking[['fold_{}'.format(fold + 1) for fold in range(folds.n_splits)]].mean(axis=1)
    feature_ranking.to_csv('feature_importances{}.csv'.format(index))
    best_iter = max(max_rds)
    lgb_clf = lgb.LGBMClassifier(**params, num_boost_round=best_iter)
    lgb_clf.fit(train_data, target)
    filename = 'finalized_model{}.sav'.format(index)
    pickle.dump(lgb_clf, open(filename, 'wb'))
    
    return lgb_clf

In [ ]:
def callPredict(lgb_clf, test_data,result_list, index):
    y_pred = lgb_clf.predict_proba(test_data)[:, 1]#convert into binary values
    result_list["isFraud"] = y_pred
    results = result_list
    results.to_csv("LGBPredictions{}.csv".format(index),index=False)

In [ ]:
for index in range(len(paramList)):
    print("Testing model {}".format(str(index+1)))
    trained_model = callTraining(index, train_data,target,folds,paramList[index])
    result_copy = result_list.copy()
    callPredict(trained_model,test_data,result_copy,index)
    f = open("params_file.txt","a")
    f.write(json.dumps(paramList[index])+'\n')
    f.close()
    plt.bar(range(len(trained_model.feature_importances_)), trained_model.feature_importances_)